In [2]:
import torch
import pandas as pd
import numpy as np
import torch.nn as nn
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn import metrics
from imblearn.over_sampling import SMOTE

In [3]:
def start_data(label, users):
    #get the dataframe column names
    name_dataframe = pd.read_csv('../data/features.txt', delimiter = '\n', header = None)
    names = name_dataframe.values.tolist()
    names = [k for row in names for k in row] #List of column names

    data = pd.read_csv('../data/X_train.txt', delim_whitespace = True, header = None) #Read in dataframe
    data.columns = names #Setting column names

    #X_train = data.loc[:,'1 tBodyAcc-mean()-X':'40 tBodyAcc-correlation()-Y,Z'] #Selecting only acceleration columns
    
    X_train_1 = data.loc[:,'1 tBodyAcc-mean()-X':'40 tBodyAcc-correlation()-Y,Z']
    X_train_2 = data.loc[:,'81 tBodyAccJerk-mean()-X':'120 tBodyAccJerk-correlation()-Y,Z']
    X_train = pd.concat([X_train_1, X_train_2], axis = 1)
    
    y_train_activity = pd.read_csv('../data/y_train.txt', header = None)
    y_train_activity.columns = ['Activity']
    
    y_train_subject = pd.read_csv('../data/subject_train.txt', header = None)
    y_train_subject.columns = ['Subject']

    GAN_data = pd.concat([X_train, y_train_activity, y_train_subject], axis = 1)
    GAN_data = GAN_data[GAN_data['Activity'].isin(label)]
    GAN_data = GAN_data[GAN_data['Subject'].isin(users)]
    
    X = GAN_data.loc[:,'1 tBodyAcc-mean()-X':'120 tBodyAccJerk-correlation()-Y,Z'].values
    y = GAN_data[['Activity']].values
    
    return X, y

In [4]:
#defines each generator layer
#input and output dimensions needed
def generator_block(input_dim, output_dim):
    return nn.Sequential(
        nn.Linear(input_dim, output_dim),
        nn.Dropout(0.1),
        nn.BatchNorm1d(output_dim),
        nn.ReLU(inplace = True)
    )

#returns n_samples of z_dim (number of dimensions of latent space) noise
def get_noise(n_samples, z_dim):
    return torch.randn(n_samples, z_dim)

#defines generator class
class Generator(nn.Module):
    def __init__(self, z_dim = 10, feature_dim = 80, hidden_dim = 128):
        super(Generator, self).__init__()
        self.gen = nn.Sequential(
            generator_block(z_dim, 95),
            generator_block(95, 90),
            generator_block(90, 85),
            nn.Linear(85, feature_dim)
        )
    def forward(self, noise):
        return self.gen(noise)
    
def get_act_matrix(batch_size, a_dim):
    indexes = np.random.randint(a_dim, size = batch_size)
    
    one_hot = np.zeros((len(indexes), indexes.max()+1))
    one_hot[np.arange(len(indexes)),indexes] = 1
    return torch.Tensor(indexes).long(), torch.Tensor(one_hot)
    
def get_usr_matrix(batch_size, u_dim):
    indexes = np.random.randint(u_dim, size = batch_size)
    
    one_hot = np.zeros((indexes.size, indexes.max()+1))
    one_hot[np.arange(indexes.size),indexes] = 1
    return torch.Tensor(indexes).long(), torch.Tensor(one_hot)

def load_model(model, model_name):
    model.load_state_dict(torch.load(f'../saved_models/{model_name}'))

In [5]:
activities = [1, 3, 4]
users = [1, 3, 5]
X, y = start_data(activities, users)

gen = Generator(z_dim = 106)
load_model(gen, "cGAN_UCI_30k_gen.param")

# Train on real, test on real

In [6]:
#Transforming activity labels (1, 3, 4 --> 0, 1, 2)
for k in range(len(y)): 
    if y[k] == 1:
        y[k] = 0
    elif y[k] == 3:
        y[k] = 1
    else:
        y[k] = 2

## Activity

In [7]:
X_train, X_test, y_train, y_test = train_test_split(X, y.flatten(), test_size = 0.2, shuffle = True)

classifier_real = LogisticRegression(penalty = 'l2', C = 0.7)
classifier_real.fit(X_train, y_train)

y_pred = classifier_real.predict(X_test)
print(metrics.classification_report(y_test, y_pred, digits = 3)) 

              precision    recall  f1-score   support

           0      1.000     1.000     1.000        46
           1      1.000     1.000     1.000        29
           2      1.000     1.000     1.000        25

    accuracy                          1.000       100
   macro avg      1.000     1.000     1.000       100
weighted avg      1.000     1.000     1.000       100



## User

In [8]:
def start_data_subject(label, users):
    #get the dataframe column names
    name_dataframe = pd.read_csv('../data/features.txt', delimiter = '\n', header = None)
    names = name_dataframe.values.tolist()
    names = [k for row in names for k in row] #List of column names

    data = pd.read_csv('../data/X_train.txt', delim_whitespace = True, header = None) #Read in dataframe
    data.columns = names #Setting column names

    #X_train = data.loc[:,'1 tBodyAcc-mean()-X':'40 tBodyAcc-correlation()-Y,Z'] #Selecting only acceleration columns
    
    X_train_1 = data.loc[:,'1 tBodyAcc-mean()-X':'40 tBodyAcc-correlation()-Y,Z']
    X_train_2 = data.loc[:,'81 tBodyAccJerk-mean()-X':'120 tBodyAccJerk-correlation()-Y,Z']
    X_train = pd.concat([X_train_1, X_train_2], axis = 1)

    y_train_activity = pd.read_csv('../data/y_train.txt', header = None)
    y_train_activity.columns = ['Activity']

    y_train_subject = pd.read_csv('../data/subject_train.txt', header = None)
    y_train_subject.columns = ['Subject']

    GAN_data = pd.concat([X_train, y_train_activity, y_train_subject], axis = 1)
    GAN_data = GAN_data[GAN_data['Activity'].isin(label)]
    GAN_data = GAN_data[GAN_data['Subject'].isin(users)]
    
    X = GAN_data.loc[:,'1 tBodyAcc-mean()-X':'120 tBodyAccJerk-correlation()-Y,Z'].values
    y = GAN_data[['Subject']].values
    
    return X, y

In [9]:
X, y_subject = start_data_subject(activities, users)
for k in range(len(y_subject)): 
    if y_subject[k] == 1:
        y_subject[k] = 0
    elif y_subject[k] == 3:
        y_subject[k] = 1
    else:
        y_subject[k] = 2

In [13]:
X_train_sub, X_test_sub, y_train_sub, y_test_sub = train_test_split(X, y_subject.flatten(), test_size = 0.2, shuffle = True)

classifier_real_user = LogisticRegression(penalty = 'l2', C = 0.7, max_iter = 200)
classifier_real_user.fit(X_train_sub, y_train_sub)

y_pred = classifier_real_user.predict(X_test_sub)
print(metrics.classification_report(y_test_sub, y_pred, digits = 3)) 

              precision    recall  f1-score   support

           0      0.976     0.909     0.941        44
           1      0.824     0.800     0.812        35
           2      0.720     0.857     0.783        21

    accuracy                          0.860       100
   macro avg      0.840     0.855     0.845       100
weighted avg      0.869     0.860     0.863       100



# Train on fake, test on real

## Activity

In [19]:
latent_vectors = get_noise(len(X_train), 100)
act_vectors = get_act_matrix(len(X_train), 3)
usr_vectors = get_usr_matrix(len(X_train), 3)

to_gen = torch.cat((latent_vectors, act_vectors[1], usr_vectors[1]), 1)
fake_features = gen(to_gen).detach().numpy()

classifier_fake = LogisticRegression(penalty = 'l2', C = 0.7)
classifier_fake.fit(fake_features, act_vectors[0])

y_pred = classifier_fake.predict(X_test)
print(metrics.confusion_matrix(y_test, y_pred))
print(metrics.classification_report(y_test, y_pred, digits = 3)) 

[[43  3  0]
 [ 1 28  0]
 [ 0  0 25]]
              precision    recall  f1-score   support

           0      0.977     0.935     0.956        46
           1      0.903     0.966     0.933        29
           2      1.000     1.000     1.000        25

    accuracy                          0.960       100
   macro avg      0.960     0.967     0.963       100
weighted avg      0.961     0.960     0.960       100



## User

In [20]:
classifier_fake_user = LogisticRegression(penalty = 'l2', C = 0.7)
classifier_fake_user.fit(fake_features, usr_vectors[0])

y_pred = classifier_fake.predict(X_test_sub)
print(metrics.confusion_matrix(y_test_sub, y_pred))
print(metrics.classification_report(y_test_sub, y_pred, digits = 3))

[[18 15 11]
 [15  9 11]
 [ 6 12  3]]
              precision    recall  f1-score   support

           0      0.462     0.409     0.434        44
           1      0.250     0.257     0.254        35
           2      0.120     0.143     0.130        21

    accuracy                          0.300       100
   macro avg      0.277     0.270     0.273       100
weighted avg      0.316     0.300     0.307       100



# Train on real + fake, test on real

In [8]:
X, y = start_data(activities, users)

In [9]:
X_train_real, X_test_real, y_train_real, y_test_real = train_test_split(X, y, test_size = 0.2, shuffle = True)

y_real = np.ones(len(X_train_real))
y_fake = np.zeros(len(fake_features))

In [10]:
X_train_combo = np.concatenate((fake_features, X_train_real), axis = 0)
y_train_combo = np.concatenate((y_fake, y_real), axis = 0)

In [11]:
def shuffle(X, y):
    data = np.concatenate((X, y.reshape(-1,1)), axis = 1)
    np.random.shuffle(data)
    return data[:,:-1], data[:,-1]

In [12]:
X_train_combo, y_train_combo = shuffle(X_train_combo, y_train_combo)

In [13]:
for i in range(0, 80):
    print(i)
    classifier = LogisticRegression(penalty = 'l2', C = 0.7)
    classifier.fit(X_train_combo[:,i].reshape(-1,1), y_train_combo)
    y_pred = classifier.predict(X_test_real[:,i].reshape(-1,1))

    y_test_real = np.ones(len(y_test_real))

    print(metrics.confusion_matrix(y_test_real, y_pred))
    print(metrics.classification_report(y_test_real, y_pred, digits = 3))     

0
[[ 0  0]
 [ 2 98]]
              precision    recall  f1-score   support

         0.0      0.000     0.000     0.000         0
         1.0      1.000     0.980     0.990       100

    accuracy                          0.980       100
   macro avg      0.500     0.490     0.495       100
weighted avg      1.000     0.980     0.990       100

1
[[100]]
              precision    recall  f1-score   support

         1.0      1.000     1.000     1.000       100

    accuracy                          1.000       100
   macro avg      1.000     1.000     1.000       100
weighted avg      1.000     1.000     1.000       100

2
[[ 0  0]
 [39 61]]
              precision    recall  f1-score   support

         0.0      0.000     0.000     0.000         0
         1.0      1.000     0.610     0.758       100

    accuracy                          0.610       100
   macro avg      0.500     0.305     0.379       100
weighted avg      1.000     0.610     0.758       100

3
[[ 0  0]
 [29 71]]


C:\Users\hkimr\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\hkimr\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\hkimr\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\hkimr\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1

[[ 0  0]
 [41 59]]
              precision    recall  f1-score   support

         0.0      0.000     0.000     0.000         0
         1.0      1.000     0.590     0.742       100

    accuracy                          0.590       100
   macro avg      0.500     0.295     0.371       100
weighted avg      1.000     0.590     0.742       100

30
[[ 0  0]
 [34 66]]
              precision    recall  f1-score   support

         0.0      0.000     0.000     0.000         0
         1.0      1.000     0.660     0.795       100

    accuracy                          0.660       100
   macro avg      0.500     0.330     0.398       100
weighted avg      1.000     0.660     0.795       100

31
[[ 0  0]
 [44 56]]
              precision    recall  f1-score   support

         0.0      0.000     0.000     0.000         0
         1.0      1.000     0.560     0.718       100

    accuracy                          0.560       100
   macro avg      0.500     0.280     0.359       100
weighted av

C:\Users\hkimr\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\hkimr\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\hkimr\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\hkimr\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1

[[ 0  0]
 [71 29]]
              precision    recall  f1-score   support

         0.0      0.000     0.000     0.000         0
         1.0      1.000     0.290     0.450       100

    accuracy                          0.290       100
   macro avg      0.500     0.145     0.225       100
weighted avg      1.000     0.290     0.450       100

62
[[ 0  0]
 [22 78]]
              precision    recall  f1-score   support

         0.0      0.000     0.000     0.000         0
         1.0      1.000     0.780     0.876       100

    accuracy                          0.780       100
   macro avg      0.500     0.390     0.438       100
weighted avg      1.000     0.780     0.876       100

63
[[ 0  0]
 [22 78]]
              precision    recall  f1-score   support

         0.0      0.000     0.000     0.000         0
         1.0      1.000     0.780     0.876       100

    accuracy                          0.780       100
   macro avg      0.500     0.390     0.438       100
weighted av

C:\Users\hkimr\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\hkimr\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\hkimr\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\hkimr\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1

In [14]:
classifier = LogisticRegression(penalty = 'l2', C = 0.7)
classifier.fit(X_train_combo, y_train_combo)
y_pred = classifier.predict(X_test_real)

y_test_real = np.ones(len(y_test_real))

print(metrics.confusion_matrix(y_test_real, y_pred))
print(metrics.classification_report(y_test_real, y_pred, digits = 3)) 

[[100]]
              precision    recall  f1-score   support

         1.0      1.000     1.000     1.000       100

    accuracy                          1.000       100
   macro avg      1.000     1.000     1.000       100
weighted avg      1.000     1.000     1.000       100

